In [71]:
from urllib.request import urlopen, Request

url = "https://db.netkeiba.com/race/202405050511"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}

request = Request(url, headers=headers)
html = urlopen(request).read()

In [1]:
html

NameError: name 'html' is not defined

In [73]:
import pandas as pd
df = pd.read_html(html)
df

[   着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気  \
 0    1    4    4   クロワデュノール  牡2  56  北村友一  1:46.8    NaN    2.2    1   
 1    2    8    9  サトノシャイニング  牡2  56  松山弘平  1:46.9    3/4    5.7    3   
 2    3    7    7   レッドキングリー  牡2  56  ルメール  1:47.1  1.1/2    2.7    2   
 3    4    3    3  ファイアンクランツ  牡2  56  菅原明良  1:47.2     クビ    8.0    4   
 4    5    5    5     デルアヴァー  牡2  56  三浦皇成  1:47.3    1/2   16.5    5   
 5    6    2    2    ジーティーマン  牡2  56  西村淳也  1:47.7  2.1/2   19.7    6   
 6    7    6    6    プレシャスデイ  牡2  56  横山武史  1:47.7    アタマ   35.9    7   
 7    8    1    1    ニシノタンギー  牡2  56   武藤雅  1:48.0  1.1/2  236.6    9   
 8    9    8    8  ニシノイストワール  牡2  56  田辺裕信  1:48.4  2.1/2   61.6    8   
 
         馬体重       調教師  
 0  504(+24)  [西] 斉藤崇史  
 1   492(+2)  [西] 杉山晴紀  
 2   504(+4)  [東] 木村哲也  
 3  454(+14)   [東] 堀宣行  
 4  478(+14)  [西] 松永幹夫  
 5  486(+12)   [西] 小栗実  
 6   490(+4)  [東] 伊坂重信  
 7   460(-2)  [東] 水野貴広  
 8   456(+4)  [東] 鹿戸雄一  ,
     0      1    

In [74]:
df[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,4,4,クロワデュノール,牡2,56,北村友一,1:46.8,NaN,2.2,1,504(+24),[西] 斉藤崇史
1,2,8,9,サトノシャイニング,牡2,56,松山弘平,1:46.9,3/4,5.7,3,492(+2),[西] 杉山晴紀
2,3,7,7,レッドキングリー,牡2,56,ルメール,1:47.1,1.1/2,2.7,2,504(+4),[東] 木村哲也
3,4,3,3,ファイアンクランツ,牡2,56,菅原明良,1:47.2,クビ,8.0,4,454(+14),[東] 堀宣行
4,5,5,5,デルアヴァー,牡2,56,三浦皇成,1:47.3,1/2,16.5,5,478(+14),[西] 松永幹夫
5,6,2,2,ジーティーマン,牡2,56,西村淳也,1:47.7,2.1/2,19.7,6,486(+12),[西] 小栗実
6,7,6,6,プレシャスデイ,牡2,56,横山武史,1:47.7,アタマ,35.9,7,490(+4),[東] 伊坂重信
7,8,1,1,ニシノタンギー,牡2,56,武藤雅,1:48.0,1.1/2,236.6,9,460(-2),[東] 水野貴広
8,9,8,8,ニシノイストワール,牡2,56,田辺裕信,1:48.4,2.1/2,61.6,8,456(+4),[東] 鹿戸雄一


In [75]:
df[1]

,0,1,2,3
0,単勝,4,220,1
1,複勝,4 9 7,110 130 110,2 3 1
2,枠連,4 - 8,550,2
3,馬連,4 - 9,660,3


In [76]:
df[2]

,0,1,2,3
0,ワイド,4 - 9 4 - 7 7 - 9,250 170 220,3 1 2
1,馬単,4 → 9,990,4
2,三連複,4 - 7 - 9,480,1
3,三連単,4 → 9 → 7,2300,4


# 開催日一覧を取得
```
https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year=2023
https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105
```

In [77]:
url = "https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year=2023"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}
request = Request(url, headers=headers)
html = urlopen(request).read()
html

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n<!-- block=common__meta_tag_common_race_schedule (d) -->\n<meta http-equiv="content-language" content="ja">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n<meta name="viewport" content="width=1000">\n<meta name="format-detection" content="telephone=no" />\n<title>\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8 | \xa5\xec\xa1\xbc\xa5\xb9\xbe\xf0\xca\xf3(JRA) - netkeiba</title>\n<meta name="description" content="\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8\xa5\xda\xa1\xbc\xa5\xb8\xa4\xc7\xa4\xb9\xa1\xa3\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xe4\xba\xc7\xbf\xb7\xa5\xaa\xa5\xc3\xa5\xba\xa1\xa2\xa5\xec\xa1\xbc\xa5\xb9\xb7\xeb\xb2\xcc\xc2\xae\xca\xf3\xa1\xa2\xca\xa7\xcc\xe1\xbe\xf0\xca\xf3\xa4\xf2\xa4\xcf\xa4\xb8\xa4\xe1\xa1\xa2\xb6\xa5\xc7\xcf\xcd\xbd\xc1\xdb\xa4\

In [78]:
# htmlから要素を取り出したい時はBeutifulsoupをつかう 

from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_common_race_schedule (d) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>開催日程 | レース情報(JRA) - netkeiba</title>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,開催日程,スケジュール,開催日,netkeiba,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba_logo_keiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="開催日程 | レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/top/calendar.html" property="og:url"/>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最

In [79]:
# aタグのhref要素から開催日を取り出す
a_tag = soup.find("table", class_="Calendar_Table").find("a")
a_tag["href"]

'../top/race_list.html?kaisai_date=20230105'

In [80]:
# 開催日を抽出 
import re
re.findall(r"kaisai_date=(\d{8})", a_tag["href"])[0]

'20230105'

In [81]:
# a_tag_list = soup.find("table", class_="Calendar_Table").find_all("a")
# kaisai_date_list = []
# for a_tag in a_tag_list:
#   kaisai_date = re.findall(r"kaisai_date=(\d{8})", a_tag["href"])[0]
#   kaisai_date_list.append(kaisai_date)

# kaisai_date_list


In [82]:
def url_open_with_headers(url):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}
  request = Request(url, headers=headers)
  html = urlopen(request).read()
  return html

In [83]:
import time
from tqdm import tqdm

def scrape_kaisai_date(from_, to_):
  kaisai_date_list = []
  for date in tqdm(pd.date_range(from_, to_, freq="MS")):
    year, month = date.year, date.month
    url = f"https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year={year}&month={month}"
    html = url_open_with_headers(url)
    time.sleep(1) # 忘れない
    soup = BeautifulSoup(html)
    a_tag_list = soup.find("table", class_="Calendar_Table").find_all("a")
    
    for a_tag in a_tag_list:
      kaisai_date = re.findall(r"kaisai_date=(\d{8})", a_tag["href"])[0]
      kaisai_date_list.append(kaisai_date)

  return kaisai_date_list


In [84]:
# from_, to_ = "2023-01", "2023-12"

# kaisai_date_list_at_2023 = scrape_kaisai_date("2023-01", "2023-12")
# kaisai_date_list_at_2023

## 開催ページからIDを取得

In [85]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"

In [86]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [87]:
html = url_open_with_headers(url)
soup = BeautifulSoup(html, features="lxml")

In [88]:
soup.find("div", class_="RaceList_Box")

取得できないときはselenium, Chrome Driverを使用

In [89]:
# pip install selenium
# pip install webdriver-manager

In [90]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [95]:
# chromeのversionにあったdriverを準備する→chromeを使う前提
driver_path = ChromeDriverManager().install()
driver = webdriver.Chrome(service=Service(driver_path))

In [96]:
driver.get(url)

In [97]:
from selenium.webdriver.common.by import By

li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259fea1f1aed39624b4", element="f.2A88A60E85D15F0ABA7CDF42450991B7.d.163CB6CFD70173EC094E5F4A11A4784C.e.966")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259fea1f1aed39624b4", element="f.2A88A60E85D15F0ABA7CDF42450991B7.d.163CB6CFD70173EC094E5F4A11A4784C.e.983")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259fea1f1aed39624b4", element="f.2A88A60E85D15F0ABA7CDF42450991B7.d.163CB6CFD70173EC094E5F4A11A4784C.e.1000")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259fea1f1aed39624b4", element="f.2A88A60E85D15F0ABA7CDF42450991B7.d.163CB6CFD70173EC094E5F4A11A4784C.e.1017")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259fea1f1aed39624b4", element="f.2A88A60E85D15F0ABA7CDF42450991B7.d.163CB6CFD70173EC094E5F4A11A4784C.e.1034")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259f

In [98]:
li_0 = li_list[0]
li_0

<selenium.webdriver.remote.webelement.WebElement (session="e250e9094e1c1259fea1f1aed39624b4", element="f.2A88A60E85D15F0ABA7CDF42450991B7.d.163CB6CFD70173EC094E5F4A11A4784C.e.966")>

In [102]:
href = li_0.find_element(By.TAG_NAME, "a").get_attribute("href")
href

'https://race.netkeiba.com/race/result.html?race_id=202306010101&rf=race_list'

In [104]:
import re
rece_id = re.findall(r"race_id=(\d{12})", href)[0]
rece_id

'202306010101'

In [106]:
race_id_list = []
for li in li_list:
  href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
  race_id = re.findall(r"race_id=(\d{12})" ,href)[0]
  race_id_list.append(race_id)

race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112']

In [107]:
len(race_id_list)

24

In [108]:
driver.quit()

In [112]:
from src.scraping import scrape_kaisai_date

In [115]:
race_date_list = scrape_kaisai_date(from_="2023-01", to_="2023-12")
race_date_list

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:18<00:00,  1.54s/it]


['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129',
 '20230204',
 '20230205',
 '20230211',
 '20230212',
 '20230218',
 '20230219',
 '20230225',
 '20230226',
 '20230304',
 '20230305',
 '20230311',
 '20230312',
 '20230318',
 '20230319',
 '20230325',
 '20230326',
 '20230401',
 '20230402',
 '20230408',
 '20230409',
 '20230415',
 '20230416',
 '20230422',
 '20230423',
 '20230429',
 '20230430',
 '20230506',
 '20230507',
 '20230513',
 '20230514',
 '20230520',
 '20230521',
 '20230527',
 '20230528',
 '20230603',
 '20230604',
 '20230610',
 '20230611',
 '20230617',
 '20230618',
 '20230624',
 '20230625',
 '20230701',
 '20230702',
 '20230708',
 '20230709',
 '20230715',
 '20230716',
 '20230722',
 '20230723',
 '20230729',
 '20230730',
 '20230805',
 '20230806',
 '20230812',
 '20230813',
 '20230819',
 '20230820',
 '20230826',
 '20230827',
 '20230902',
 '20230903',
 '20230909',
 '20230910',
 '20230916',
 '20230917',
 '20230918',

In [ ]:
import time
from tqdm import tqdm
import traceback
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


def scrape_race_ids(race_date_list: list[str]):
  driver_options = Options()
  driver_options.add_argument("--headless") # ヘッドレスで実行
  driver_path = ChromeDriverManager().install()
  race_id_list = []
  with webdriver.Chrome(service=Service(driver_path), options=driver_options) as driver:
  # driver = webdriver.Chrome(service=Service(driver_path), options=driver_options)
    for race_date in tqdm(race_date_list):
      url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={race_date}"
      try:
        driver.get(url)
        time.sleep(1) # 忘れない
        li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
        
        for li in li_list:
          href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
          race_id = re.findall(r"race_id=(\d{12})" ,href)[0]
          race_id_list.append(race_id)
      except:
        print("stop at", url)
        print(traceback.format_exc())
        break
    
  # driver.quit()
  return race_id_list

In [5]:
from src.scraping import scrape_kaisai_date
race_date_list = race_date_list = scrape_kaisai_date(from_="2023-01", to_="2023-02")
race_id_list = scrape_race_ids(race_date_list)

race_id_list

  0%|          | 0/18 [00:00<?, ?it/s]

['20230105', '20230107', '20230108', '20230109', '20230114', '20230115', '20230121', '20230122', '20230128', '20230129', '20230204', '20230205', '20230211', '20230212', '20230218', '20230219', '20230225', '20230226'] [<selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.C93B47969F4686E31841AA4449C6EB6D.e.84")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.C93B47969F4686E31841AA4449C6EB6D.e.85")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.C93B47969F4686E31841AA4449C6EB6D.e.86")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.C93B47969F4686E31841AA4449C6EB6D.e.87")>, <selenium.webdriver.remote.webelement.WebElement (sess

  6%|▌         | 1/18 [01:37<27:42, 97.81s/it]

['20230105', '20230107', '20230108', '20230109', '20230114', '20230115', '20230121', '20230122', '20230128', '20230129', '20230204', '20230205', '20230211', '20230212', '20230218', '20230219', '20230225', '20230226'] [<selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.F1B9A8E786FF39F414A74A2518FD11FA.e.180")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.F1B9A8E786FF39F414A74A2518FD11FA.e.181")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.F1B9A8E786FF39F414A74A2518FD11FA.e.182")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.F1B9A8E786FF39F414A74A2518FD11FA.e.183")>, <selenium.webdriver.remote.webelement.WebElement (

 11%|█         | 2/18 [01:56<13:37, 51.12s/it]

...

['20230105', '20230107', '20230108', '20230109', '20230114', '20230115', '20230121', '20230122', '20230128', '20230129', '20230204', '20230205', '20230211', '20230212', '20230218', '20230219', '20230225', '20230226'] [<selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.CD75E5477759A0C9E7B4F148D10BECAC.e.2120")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.CD75E5477759A0C9E7B4F148D10BECAC.e.2121")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.CD75E5477759A0C9E7B4F148D10BECAC.e.2122")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.CD75E5477759A0C9E7B4F148D10BECAC.e.2123")>, <selenium.webdriver.remote.webelement.WebEleme

 94%|█████████▍| 17/18 [06:50<00:47, 47.57s/it]

['20230105', '20230107', '20230108', '20230109', '20230114', '20230115', '20230121', '20230122', '20230128', '20230129', '20230204', '20230205', '20230211', '20230212', '20230218', '20230219', '20230225', '20230226'] [<selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.BDAF01EA74212B6B4289ACDA3BBE1C8E.e.2240")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.BDAF01EA74212B6B4289ACDA3BBE1C8E.e.2241")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.BDAF01EA74212B6B4289ACDA3BBE1C8E.e.2242")>, <selenium.webdriver.remote.webelement.WebElement (session="64dc78913f524a71460eaaec334db76b", element="f.D9965D75F995833E0F702563E50B72F3.d.BDAF01EA74212B6B4289ACDA3BBE1C8E.e.2243")>, <selenium.webdriver.remote.webelement.WebEleme

100%|██████████| 18/18 [07:05<00:00, 23.66s/it]


['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112',
 '202306010201',
 '202306010202',
 '202306010203',
 '202306010204',
 '202306010205',
 '202306010206',
 '202306010207',
 '202306010208',
 '202306010209',
 '202306010210',
 '202306010211',
 '202306010212',
 '202307010201',
 '202307010202',
 '202307010203',
 '202307010204',
 '202307010205',
 '202307010206',
 '202307010207',
 '202307010208',
 '202307010209',
 '202307010210',
 '202307010211',
 '202307010212',
 '202306010301',
 '202306010302',
 '202306010303',
 '202306010304',
 '202306010305',
 '202306010306',
 '202306010307',
 '202306010308',
 '202306010309',
 '202306010310',
 '202306010311

### レース詳細画面からテーブルを取得

In [7]:
from src.scraping import url_open_with_headers


In [8]:
url = "https://db.netkeiba.com/race/202306010101"
html = url_open_with_headers(url)

In [ ]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>3\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32023\xc7\xaf1\xb7\xee5\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\

In [ ]:
import pandas as pd

In [13]:
table = pd.read_html(html)
table[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,7,14,シュバルツガイスト,牡3,56,武豊,1:12.6,NaN,1.8,1,486(-4),[東] 国枝栄
1,2,3,5,ジェイケイファイン,牡3,56,内田博幸,1:12.9,1.3/4,7.4,3,472(-8),[東] 奥平雅士
2,3,1,1,レディアス,牝3,54,津村明秀,1:12.9,クビ,36.1,6,428(+6),[東] 鈴木慎太
3,4,8,15,ハローマイラブリー,牝3,54,三浦皇成,1:13.4,3,4.2,2,536(0),[東] 伊坂重信
4,5,5,9,ベアカワイコチャン,牝3,54,松岡正海,1:13.9,3,7.9,4,478(+10),[東] 杉浦宏昭
5,6,1,2,ラディーヴァ,牝3,52,原優介,1:14.0,1/2,255.6,13,456(-2),[東] 武市康男
6,7,6,12,メルティーショコラ,牝3,54,武藤雅,1:14.0,クビ,58.8,7,470(+14),[東] 武藤善則
7,8,8,16,ニシノスイセイ,牡3,56,野中悠太,1:14.1,1/2,137.1,11,468(+6),[東] 青木孝文
8,9,7,13,タッカーカムカム,牝3,52,山田敬士,1:14.2,クビ,322.0,14,426(-2),[東] 鈴木伸尋
9,10,3,6,ペイシャコリン,牝3,52,小林凌大,1:14.4,1.1/2,86.0,10,454(-10),[東] 伊藤伸一


In [27]:
import pickle

with open("src/race_id_list_2023.pkl", "rb") as f:
  race_id_list = pickle.load(f)
  

In [28]:
len(race_id_list)

3456

In [ ]:
# import time

# for race_id in race_id_list[:4]:
#   url = f"https://db.netkeiba.com/race/{race_id}"
#   html = url_open_with_headers(url)
#   time.sleep(1)
#   df = pd.read_html(html)[0]



`df`を`dump`するの前に、後々、データ追加したくなる時用にHTML自体を保存する

In [32]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>4\xba\xd0\xb0\xca\xbe\xe51\xbe\xa1\xa5\xaf\xa5\xe9\xa5\xb9\xa1\xc32023\xc7\xaf1\xb7\xee5\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1

In [33]:
with open("data/html/202306010101.bin", "wb") as f:
  f.write(html)

In [36]:
from pathlib import Path

HTML_DIR = Path("data", "html")

In [37]:
with open(HTML_DIR / "202306010101.bin", "wb") as f:
  f.write(html)

In [42]:
import time
from pathlib import Path
from tqdm import tqdm

HTML_DIR = Path("data", "html")

def scrape_race_detail_html(race_id_list: list[str], save_dir: Path) -> list[Path]:
  html_path_list = []
  for race_id in tqdm(race_id_list):
    # file exists check
    filepath = save_dir / f"{race_id}.bin"
    if filepath.is_file():
      print("skipped", filepath)
      continue
    
    # scraping
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = url_open_with_headers(url)
    time.sleep(1)
    
    # write html
    with open(filepath, "wb") as f:
      f.write(html)
    html_path_list.append(filepath)
  
  return html_path_list

In [45]:
html_path_list = scrape_race_detail_html(race_id_list[:12], HTML_DIR)
html_path_list

100%|██████████| 12/12 [00:22<00:00,  1.83s/it]


[PosixPath('data/html/202306010101.bin'),
 PosixPath('data/html/202306010102.bin'),
 PosixPath('data/html/202306010103.bin'),
 PosixPath('data/html/202306010104.bin'),
 PosixPath('data/html/202306010105.bin'),
 PosixPath('data/html/202306010106.bin'),
 PosixPath('data/html/202306010107.bin'),
 PosixPath('data/html/202306010108.bin'),
 PosixPath('data/html/202306010109.bin'),
 PosixPath('data/html/202306010110.bin'),
 PosixPath('data/html/202306010111.bin'),
 PosixPath('data/html/202306010112.bin')]

In [49]:
import os
from src.constants import PROJECT_ROOT, RACE_DETAIL_HTML_DIR

os.chdir(PROJECT_ROOT)
print("Current working directory:", os.getcwd())

Current working directory: /Users/fumiya/dev/python/keiba


In [55]:
print(RACE_DETAIL_HTML_DIR)

/Users/fumiya/dev/python/keiba/data/html/race


In [69]:
html_path_list = list(RACE_DETAIL_HTML_DIR.glob("*"))

In [57]:
len(html_path_list)

1998

In [66]:
dfs = {}
for html_path in html_path_list[:3]:
  with open(html_path, "rb") as f:
    race_id = html_path.stem
    html = f.read()
    df = pd.read_html(html)[0]
    df.index = [race_id] * len(df) # race_idをindexに(list * int = 繰り返しになる)
    dfs[race_id] = df
    
# valueだけ繋げる
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"


In [67]:
concat_df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202309010204,1,7,11,セラフィックコール,牡3,56,ムルザバ,1:55.2,NaN,2.3,1,540(0),[西] 寺島良
202309010204,2,8,13,レッドプロフェシー,牡3,56,鮫島克駿,1:56.5,8,11.2,5,498(0),[西] 大久保龍
202309010204,3,1,1,チュウワハート,牡3,56,武豊,1:56.7,1.1/4,3.0,2,514(0),[西] 大久保龍
202309010204,4,4,5,オネストリー,牡3,56,横山武史,1:56.8,1/2,24.3,6,468(0),[西] 茶木太樹
202309010204,5,7,10,エンツォウーノ,牡3,56,荻野琢真,1:56.9,1,10.9,4,498(0),[西] 田中克典
202309010204,6,5,7,ホペロア,牡3,56,坂井瑠星,1:56.9,ハナ,5.7,3,522(0),[西] 友道康夫
202309010204,7,4,4,カワキタハクギン,牡3,53,川端海翼,1:57.5,3.1/2,33.6,7,520(0),[西] 浜田多実
202309010204,8,6,9,ステラート,牡3,56,水口優也,1:59.4,大,61.1,8,482(0),[西] 宮本博
202309010204,9,5,6,エピックライズ,牡3,52,永島まな,1:59.6,1.1/4,72.6,10,444(0),[西] 高柳大輔


In [ ]:
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
  dfs = {}
  for html_path in tqdm(html_path_list):
    with open(html_path, "rb") as f:
      race_id = html_path.stem
      html = f.read()
      df = pd.read_html(html)[0]
      df.index = [race_id] * len(df) # race_idをindexに(list * int = 繰り返しになる)
      dfs[race_id] = df
    
  # valueだけ繋げる
  concat_df = pd.concat(dfs.values())
  concat_df.index.name = "race_id"
  return concat_df

In [80]:
results = create_results(html_path_list)

In [81]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202309010204,1,7,11,セラフィックコール,牡3,56.0,ムルザバ,1:55.2,NaN,2.3,1.0,540(0),[西] 寺島良
202309010204,2,8,13,レッドプロフェシー,牡3,56.0,鮫島克駿,1:56.5,8,11.2,5.0,498(0),[西] 大久保龍
202309010204,3,1,1,チュウワハート,牡3,56.0,武豊,1:56.7,1.1/4,3.0,2.0,514(0),[西] 大久保龍
202309010204,4,4,5,オネストリー,牡3,56.0,横山武史,1:56.8,1/2,24.3,6.0,468(0),[西] 茶木太樹
202309010204,5,7,10,エンツォウーノ,牡3,56.0,荻野琢真,1:56.9,1,10.9,4.0,498(0),[西] 田中克典
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202309011103,12,7,13,サムシンググッド,セ3,56.0,吉田隼人,1:14.5,ハナ,29.1,10.0,424(-12),[西] 新谷功一
202309011103,13,4,7,テイエムジンリョク,牡3,56.0,太宰啓介,1:14.9,2,574.5,16.0,536(-10),[西] 木原一良
202309011103,14,8,15,ヨドノルーキー,牡3,56.0,藤岡佑介,1:15.0,3/4,20.7,8.0,404(+6),[西] 飯田雄三


## ウマの過去成績テーブルの作成

In [ ]:
# ウマの過去成績ページ
url = "https://db.netkeiba.com/horse/2020103575"

In [84]:
from src.scraping import url_open_with_headers

html = url_open_with_headers(url)
html

b'<!-- Bamei:\xa5\xb7\xa5\xe5\xa5\xd0\xa5\xeb\xa5\xc4\xa5\xac\xa5\xa4\xa5\xb9\xa5\xc8 --><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?20210819" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/db_detail.css?20180621" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/h

In [87]:
import pandas as pd

pd.read_html(html)[2]

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/12/22,5中山8,晴,8,冬至特別(2勝クラス),NaN,16,1,2,99.0,...,1.4,**,11-11,33.6-38.0,38.4,508(+12),NaN,NaN,ロードフロンティア,NaN
1,2024/08/31,2中京7,小雨,10,鳥羽特別(2勝クラス),NaN,15,3,5,32.6,...,4.6,**,10-10,34.7-36.8,40.5,496(-8),NaN,NaN,カズゴルティス,NaN
2,2024/07/27,2新潟1,曇,12,出雲崎特別(2勝クラス),NaN,15,6,10,8.0,...,0.9,**,4-4,34.2-36.7,37.3,504(+10),NaN,NaN,エランティス,NaN
3,2024/05/19,1新潟8,晴,10,火打山特別(2勝クラス),NaN,15,5,9,13.0,...,0.6,**,6-6,33.6-37.5,37.1,494(-6),NaN,NaN,シラキヌ,230.0
4,2024/05/05,1新潟4,晴,12,4歳以上1勝クラス,NaN,13,4,4,4.5,...,-0.2,**,7-7,35.2-37.6,36.9,500(+2),NaN,NaN,(セキテイオー),800.0
5,2024/02/25,1阪神2,小雨,12,4歳以上1勝クラス,NaN,15,7,13,8.6,...,0.9,**,8-10,34.6-36.4,36.3,498(0),NaN,NaN,サクハル,NaN
6,2023/12/16,5中山5,晴,12,3歳以上1勝クラス,NaN,16,4,7,4.1,...,1.4,**,2-3,34.7-36.5,37.7,498(+4),NaN,NaN,ヤングワールド,NaN
7,2023/09/24,4中山7,晴,8,3歳以上1勝クラス,NaN,16,1,2,2.9,...,1.2,**,4-4,33.2-38.2,38.9,494(-2),NaN,NaN,トモジャミ,80.0
8,2023/08/12,3新潟1,晴,12,3歳以上1勝クラス,NaN,15,8,15,5.0,...,0.6,**,3-3,34.6-36.5,36.7,496(-4),NaN,NaN,キュートヘスティア,320.0
9,2023/07/22,2福島7,晴,12,3歳以上1勝クラス,NaN,16,8,15,8.5,...,0.9,**,10-8,30.8-37.5,37.2,500(+8),NaN,NaN,シアージスト,120.0


#### raceに紐づくウマIDを取得する

In [ ]:
# 対象レースのURL
race_url = "https://db.netkeiba.com/race/202306010101/"
html = url_open_with_headers(race_url)
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>3\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32023\xc7\xaf1\xb7\xee5\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\

In [107]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "lxml")
table = soup.find("table", class_="race_table_01 nk_tb_common")

print(table)

<table cellpadding="0" cellspacing="1" class="race_table_01 nk_tb_common" summary="レース結果">
<tr class="txt_c">
<th nowrap="nowrap">着<br/>順</th>
<th nowrap="nowrap">枠<br/>番</th>
<th nowrap="nowrap">馬<br/>番</th>
<th nowrap="nowrap">馬名</th>
<th nowrap="nowrap">性齢</th>
<th nowrap="nowrap">斤量</th>
<th nowrap="nowrap">騎手</th>
<th nowrap="nowrap">タイム</th>
<th nowrap="nowrap">着差</th>
<diary_snap_cut>
<th nowrap="nowrap">ﾀｲﾑ<br/>指数<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th nowrap="nowrap">通過</th>
<th nowrap="nowrap">上り</th>
</diary_snap_cut>
<th nowrap="nowrap">単勝</th>
<th nowrap="nowrap">人<br/>気</th>
<th nowrap="nowrap">馬体重</th>
<diary_snap_cut>
<th nowrap="nowrap">調教<br/>ﾀｲﾑ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th nowrap="nowrap">厩舎<br/>ｺﾒﾝﾄ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium

#### aタグの中でhorseを含むものだけにを絞り込む

In [111]:
import re
# ^は前方一致
a_list = table.find_all("a", href=re.compile(r"^/horse/"))

URLからrace idを取り出す

In [115]:
example_url = a_list[0]["href"]
horse_id = re.findall(r"\d{10}", example_url)[0]
horse_id

'2020103575'

In [120]:
from tqdm import tqdm

horse_id_list = []
for a in tqdm(a_list):
  horse_id = re.findall(r"\d{10}", a["href"])[0]
  horse_id_list.append(horse_id)

100%|██████████| 16/16 [00:00<00:00, 22671.91it/s]


In [122]:
len(horse_id_list)

16

In [1]:
# 関数化テスト
from src.create_rawdf import create_race_results
from src.constants import RACE_DETAIL_HTML_DIR, RACE_RESULTS_RAWDF_DIR

html_path_list_2023 = list(RACE_DETAIL_HTML_DIR.glob("*.bin"))
results = create_race_results(html_path_list=html_path_list_2023[:10], 
                         save_dir=RACE_RESULTS_RAWDF_DIR, 
                         save_filename="results_2023.csv")

100%|██████████| 10/10 [00:01<00:00,  5.15it/s]


In [2]:
# results

In [6]:
# results.isnull().sum()

resultsを保存しておく

In [7]:
# 関数に組み込む
# from src.constants import RACE_RESULTS_RAWDF_DIR

# results.to_csv(RACE_RESULTS_RAWDF_DIR / "results_2023.csv", sep="\t")

### ウマID一覧を取得する

In [2]:
horse_id_list = results["horse_id"].unique()
len(horse_id_list)

140

### 馬の過去成績テーブルを作成する

In [3]:
from src.constants import HORSE_DETAIL_HTML_DIR
from src.scraping import scrape_horse_detail_html

html_path_horse = scrape_horse_detail_html(horse_id_list=horse_id_list)

100%|██████████| 140/140 [00:00<00:00, 11777.74it/s]

skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020103400.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020101579.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020102964.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020100113.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020104790.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020103557.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020100106.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020104017.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020100788.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020106236.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020106038.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020101219.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020104225.bin
skipped /Users/fumiya/dev/python/keiba/data/html/horse/2020103470.bin
skipped /Users/fumiy

In [4]:
from src.constants import HORSE_DETAIL_HTML_DIR, RACE_RESULTS_RAWDF_DIR
html_path_horse = list(HORSE_DETAIL_HTML_DIR.glob("*.bin"))
len(html_path_horse)

140

In [ ]:
from src.create_rawdf import create_horse_results

horse_results = create_horse_results(html_path_list=html_path_horse, save_dir=RACE_RESULTS_RAWDF_DIR, save_filename="horse_results_2023")

  1%|▏         | 2/140 [00:00<00:07, 18.79it/s]

100%|██████████| 140/140 [00:05<00:00, 24.50it/s]


{'2020101047':                     日付    開催 天 気   R            レース名  映 像  頭 数  枠 番  馬 番  \
2020101047  2024/10/21   名古屋   晴  10             B8組  NaN   12    1    1   
2020101047  2024/10/04   名古屋   雨   9             B6組  NaN   12    6    8   
2020101047  2024/09/18   名古屋   晴   8   YJSトライアルラウ(B)  NaN   12    3    3   
2020101047  2024/09/06   名古屋   晴   8             B8組  NaN   11    6    6   
2020101047  2024/08/23   名古屋   晴   9             B8組  NaN   12    1    1   
2020101047  2024/08/09   名古屋   晴  12       日間賀島特別(C)  NaN   11    3    3   
2020101047  2024/07/26   名古屋   晴   8             C2組  NaN   12    4    4   
2020101047  2024/06/28   名古屋   雨   9   煮え渕ポットホール特(C)  NaN   12    8   11   
2020101047  2024/06/17   名古屋   雨  12     四谷の千枚田特別(C)  NaN   12    6    8   
2020101047  2024/05/30   名古屋   晴   4             C8組  NaN   12    1    1   
2020101047  2024/05/17   名古屋   晴   4            C10組  NaN   12    7   10   
2020101047  2024/05/02   名古屋   晴   7            C13組  NaN   12    2    2 